In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='syp', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: syp...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:44


In [31]:
# dau = mysql.read_table(spark, "pubg_gi", "all_traffic_activity_daily")

In [32]:
# wau = mysql.read_table(spark, "pubg_gi", "all_traffic_activity_weekly")

wau
- startdate = "2021-09-02"
- enddate = "2021-09-08"

In [36]:
wau.where((col("startdate") == "2021-09-02") & (col("device") == "PC")).groupBy("startdate", "enddate").agg(sum("wau").alias("wau")).show()

+----------+----------+-------+
| startdate|   enddate|    wau|
+----------+----------+-------+
|2021-09-02|2021-09-08|7430010|
+----------+----------+-------+



In [37]:
wau.where((col("startdate") == "2021-09-09") & (col("device") == "PC")).groupBy("startdate", "enddate").agg(sum("wau").alias("wau")).show()

+----------+----------+-------+
| startdate|   enddate|    wau|
+----------+----------+-------+
|2021-09-09|2021-09-15|7005556|
+----------+----------+-------+



event_name = 202109_pcs5

In [3]:
user = load_data_mart(device="pc", start_date="2021-09-01", end_date="2021-09-30", table_name="user_master")

In [63]:
items = mysql.read_table(spark, "metainfo", "meta_vc_sales_items")
pcs5_items = items.where(col("event_name") == "202109_pcs5").select("product_id", "product_name").distinct().toPandas()
pcs5_product_id_list = pcs5_items.product_id.tolist()

In [4]:
gcoin = mysql.read_table(spark, "pubg_gi", "all_economy_gcoin_used_by_product_daily")

pcs5 시작 전 일주일 ARPU

In [42]:
user.where((col("date") >= "2021-09-01") & (col("date") <= "2021-09-07") & (col("lastlogindate") >= "2021-09-01")).select(countDistinct("accountid").alias("wau")).show()

+-------+
|    wau|
+-------+
|3752598|
+-------+



In [5]:
gcoin.where((col("device") == "PC") & (col("date")>="2021-09-01") & (col("date")<="2021-09-07") & (col("product_id") == "total")).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).toPandas()

,free,paid,total
0,414693.5,4230066.5,4644760.0


pcs5 시작 후 pcs5 상품 일주일 ARPU (week1)

In [46]:
user.where((col("date") >= "2021-09-08") & (col("date") <= "2021-09-14") & (col("lastlogindate") >= "2021-09-08")).select(countDistinct("accountid").alias("wau")).show()

+-------+
|    wau|
+-------+
|3500350|
+-------+



In [75]:
gcoin.where((col("date") >= "2021-09-08") & (col("date") <= "2021-09-14") & (col("device") == "PC") & (col("product_id").isin(pcs5_product_id_list))).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).toPandas()

,free,paid,total
0,503908.6,1753469.6,2257378.2


In [78]:
2257378.2/3500350

0.6449007099290072

pcs5 week2 

In [47]:
user.where((col("date") >= "2021-09-15") & (col("date") <= "2021-09-21") & (col("lastlogindate") >= "2021-09-15")).select(countDistinct("accountid").alias("wau")).show()

+-------+
|    wau|
+-------+
|3740217|
+-------+



In [84]:
gcoin.where((col("date") >= "2021-09-15") & (col("date") <= "2021-09-21") & (col("device") == "PC") & (col("product_id").isin(pcs5_product_id_list))).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).toPandas()

,free,paid,total
0,106907.7,439008.0,545915.7


In [85]:
545915.7/3740217

0.1459582960026116

pcs5 week3

In [62]:
user.where((col("date") >= "2021-09-22") & (col("date") <= "2021-09-28") & (col("lastlogindate") >= "2021-09-22")).select(countDistinct("accountid").alias("wau")).show()

+-------+
|    wau|
+-------+
|3447821|
+-------+



In [86]:
gcoin.where((col("date") >= "2021-09-22") & (col("date") <= "2021-09-28") & (col("device") == "PC") & (col("product_id").isin(pcs5_product_id_list))).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).toPandas()

,free,paid,total
0,73273.7,316024.0,389297.7


In [87]:
389297.7/3447821

0.1129112271199694

16.1 업데이트 전 일주일 ARPU

In [81]:
user2 = load_data_mart(device="pc", start_date="2022-02-02", end_date="2022-02-15", table_name="user_master")

In [82]:
user2.where((col("date") >= "2022-02-09") & (col("date") <= "2022-02-15") & (col("lastlogindate") >= "2022-02-09")).select(countDistinct("accountid").alias("wau")).show()

+-------+
|    wau|
+-------+
|6458125|
+-------+



In [6]:
gcoin.where((col("device") == "PC") & (col("date")>="2022-02-09") & (col("date")<="2022-02-15") & (col("product_id") == "total")).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).toPandas()

,free,paid,total
0,7170584.85,3178324.25,10348909.1


16.1 update week 1

In [88]:
load_data_mart(device="pc", start_date="2022-02-16", end_date="2022-02-22", table_name="user_master").where((col("lastlogindate") >= "2022-02-16")).select(countDistinct("accountid").alias("wau")).show()

no data exists in
s3a://pubg-log-labs/data_mart/user_master_ver2/2022-02-22/PC
+-------+
|    wau|
+-------+
|5450240|
+-------+



In [ ]:
gcoin.where((col("date") >= "2021-09-08") & (col("date") <= "2021-09-14") & (col("device") == "PC") & (col("product_id").isin(pcs5_product_id_list))).select((sum("free_gcoin_used")/100).alias("free"), (sum("paid_gcoin_used")/100).alias("paid"), ((sum(col("paid_gcoin_used")+col("free_gcoin_used"))/100).alias("total"))).toPandas()

pcs6 가 앞에 붙여져 있는것들은 pcs6 실제 데이터가 아닌 예측을 위한 것들

In [12]:
pcs6_dau = dau.where((col("date") >= "2022-02-09")  & (col("date") <= "2022-02-15")).groupBy("device", "date").agg(sum("dau").alias("dau")).orderBy("device", "date").toPandas()

In [13]:
pcs6_dau

,device,date,dau
0,CONSOLE,2022-02-09,358547
1,CONSOLE,2022-02-10,351334
2,CONSOLE,2022-02-11,365587
3,CONSOLE,2022-02-12,409421
4,CONSOLE,2022-02-13,411590
5,CONSOLE,2022-02-14,317772
6,CONSOLE,2022-02-15,327516
7,PC,2022-02-09,2279462
8,PC,2022-02-10,2259991
9,PC,2022-02-11,2372755
